# The Wave Equation

Let's solve the (1+1)D wave equation

$$
\partial^2_t u(t, x) = c^2 \partial^2_x u(t, x)
$$

on $x\in [0, L]$ subject to the boundary conditions

$$
u(t, 0) = 0 = u(t, L)
$$

and initial conditions

$$
u(0, x) = A(x), \quad \partial_t u(0, x) = B(x)
$$

The standard approach is to discretize the partial derivatives. We introduce equidistant numerical grids for $t$ and $x$, so
that we have

$$
t_n = n \Delta t, \quad x_i = i \Delta x
$$

In [145]:
from findiff import Stencil
from findiff.symbolics import Equation, DerivativeSymbol as D, Symbol
import findiff.symbolics as fds
import sympy as sp
from sympy import IndexedBase

In [146]:
# t-axis: 0, x-axis: 1
c = Symbol('c', real=True)
dt = Symbol(r'\Delta t', real=True)
dx = Symbol(r'\Delta x', real=True)
d2_dx2, syms = Stencil(offsets=[(0, 1), (0, 0), (0, -1)], partials=D(1, 2), spacings=(dt, dx), symbolic=True).as_expression(index_symbols=('n', 'i'))
d2_dt2, syms2 = Stencil(offsets=[(1, 0), (0, 0), (-1, 0)], partials=D(0, 2), spacings=(dt, dx), symbolic=True).as_expression(index_symbols=('n', 'i'))

In [147]:
u = IndexedBase('u')
i, n = sp.symbols('i n')

In [148]:
eq = Equation(c**2 * d2_dx2, d2_dt2)
eq

Eq(c**2*(u[n, i + 1]/\Delta x**2 + u[n, i - 1]/\Delta x**2 - 2*u[n, i]/\Delta x**2), u[n + 1, i]/\Delta t**2 + u[n - 1, i]/\Delta t**2 - 2*u[n, i]/\Delta t**2)

In [149]:
_ * dx**2

Eq(\Delta x**2*c**2*(u[n, i + 1]/\Delta x**2 + u[n, i - 1]/\Delta x**2 - 2*u[n, i]/\Delta x**2), \Delta x**2*(u[n + 1, i]/\Delta t**2 + u[n - 1, i]/\Delta t**2 - 2*u[n, i]/\Delta t**2))

In [150]:
_.expand()

Eq(c**2*u[n, i + 1] + c**2*u[n, i - 1] - 2*c**2*u[n, i], \Delta x**2*u[n + 1, i]/\Delta t**2 + \Delta x**2*u[n - 1, i]/\Delta t**2 - 2*\Delta x**2*u[n, i]/\Delta t**2)

In [151]:
fds.collect(_, dx**2/dt**2)

Eq(c**2*u[n, i + 1] + c**2*u[n, i - 1] - 2*c**2*u[n, i], \Delta x**2*(u[n + 1, i] + u[n - 1, i] - 2*u[n, i])/\Delta t**2)

In [152]:
_* c**-2

Eq((c**2*u[n, i + 1] + c**2*u[n, i - 1] - 2*c**2*u[n, i])/c**2, \Delta x**2*(u[n + 1, i] + u[n - 1, i] - 2*u[n, i])/(\Delta t**2*c**2))

In [153]:
_.simplify()

Eq(u[n, i + 1] + u[n, i - 1] - 2*u[n, i], \Delta x**2*(u[n + 1, i] + u[n - 1, i] - 2*u[n, i])/(\Delta t**2*c**2))

In [154]:
eq = _

In [155]:
C = Symbol('C')
eq_C = Equation(C, dt * c / dx)
eq_C

Eq(C, \Delta t*c/\Delta x)

In [156]:
eq.subs(eq_C.solve(dx).as_subs())

Eq(u[n, i + 1] + u[n, i - 1] - 2*u[n, i], (u[n + 1, i] + u[n - 1, i] - 2*u[n, i])/C**2)

In [157]:
_.solve(u[n+1, i])

Eq(u[n + 1, i], C**2*u[n, i + 1] + C**2*u[n, i - 1] - 2*C**2*u[n, i] - u[n - 1, i] + 2*u[n, i])

In [158]:
eq = fds.collect(_, C**2)
eq

Eq(u[n + 1, i], C**2*(u[n, i + 1] + u[n, i - 1] - 2*u[n, i]) - u[n - 1, i] + 2*u[n, i])

In [159]:
eq.subs(n, 0)

Eq(u[1, i], C**2*(u[0, i + 1] + u[0, i - 1] - 2*u[0, i]) - u[-1, i] + 2*u[0, i])

Here we have a term $u_{-1, i}$ which is outside of the $t$-grid. Fortunately, we can
replace this by making use of the initial condition $\partial_t u(0, x) = V(x)$:

In [160]:
d_dt = Stencil(offsets=[(-1, 0), (0, 0), (1, 0)], partials=D(0), spacings=[dt, dx], symbolic=True)
d_dt, syms = d_dt.as_expression(index_symbols=('n', 'i'))
d_dt

u[n + 1, i]/(2*\Delta t) - u[n - 1, i]/(2*\Delta t)

In [161]:
B = IndexedBase('B')
eq_ini = Equation(d_dt, B[i]).subs(n, 0)
eq_ini

Eq(-u[-1, i]/(2*\Delta t) + u[1, i]/(2*\Delta t), B[i])

In [162]:
eq_ini = eq_ini.solve(u[-1, i])
eq_ini

Eq(u[-1, i], -2*\Delta t*B[i] + u[1, i])

In [163]:
eq.subs(n, 0).subs(eq_ini.as_subs())

Eq(u[1, i], C**2*(u[0, i + 1] + u[0, i - 1] - 2*u[0, i]) + 2*\Delta t*B[i] + 2*u[0, i] - u[1, i])

In [164]:
_.simplify()

Eq(u[1, i], C**2*(u[0, i + 1] + u[0, i - 1] - 2*u[0, i]) + 2*\Delta t*B[i] + 2*u[0, i] - u[1, i])

In [84]:
_.solve(u[1, i])

Eq(u[1, i], C**2*u[0, i + 1]/2 + C**2*u[0, i - 1]/2 - C**2*u[0, i] + \Delta t*V[i] + u[0, i])

In [85]:
fds.collect(_, C**2)

Eq(u[1, i], C**2*(u[0, i + 1]/2 + u[0, i - 1]/2 - u[0, i]) + \Delta t*V[i] + u[0, i])

In [141]:
import numpy as np

L = 1
nt, nx = 1000, 30
x = np.linspace(0, L, nx)
t = np.linspace(0, 1, nt)
dx = x[1] - x[0]
dt = t[1] - t[0]
c = 1
C = c * dt / dx

u = np.zeros((nt, nx), dtype=float) # implicitly satisfies boundary conditions

# Initial conditions
A = x * (L-x)
B = np.zeros_like(x)

u[0, :] = A

# First step, implementing second initial condition
u[1, 1:-1] = C**2 /2 * (u[0, 2:] + u[0, :-2] - 2*u[0, 1:-1]) + dt * B[1:-1] + A[1:-1]

# Now all other time steps
for n in range(1, nt-1):
    u[n+1, 1:-1] = C**2 * (u[n, 2:] + u[n, :-2] - 2*u[n, 1:-1]) - u[n-1, 1:-1] + 2*u[n, 1:-1]


In [165]:
# t-axis: 0, x-axis: 1
c = Symbol('c', real=True)
dt = Symbol(r'\Delta t', real=True)
dx = Symbol(r'\Delta x', real=True)
d2_dx2, syms = Stencil(offsets=[(0, 1), (0, 0), (0, -1)], partials=D(1, 2), spacings=(dt, dx),
                       symbolic=True).as_expression(index_symbols=('n', 'i'))
d2_dt2, syms2 = Stencil(offsets=[(1, 0), (0, 0), (-1, 0)], partials=D(0, 2), spacings=(dt, dx),
                        symbolic=True).as_expression(index_symbols=('n', 'i'))
u = IndexedBase('u')
i, n = sp.symbols('i n')
eq = Equation(c ** 2 * d2_dx2, d2_dt2)
eq

Eq(c**2*(u[n, i + 1]/\Delta x**2 + u[n, i - 1]/\Delta x**2 - 2*u[n, i]/\Delta x**2), u[n + 1, i]/\Delta t**2 + u[n - 1, i]/\Delta t**2 - 2*u[n, i]/\Delta t**2)

Now we are using Neumann boundary conditions

In [167]:
stc = Stencil(offsets=[(0, 0), (0, 1), (0, 2)], partials=D(1), spacings=(dt, dx), symbolic=True)
d_dx, syms = stc.as_expression(index_symbols=('n', 'i'))
d_dx.subs(i, 0)

-3*u[n, 0]/(2*\Delta x) + 2*u[n, 1]/\Delta x - u[n, 2]/(2*\Delta x)